# Melatih RL untuk Menyeimbangkan Cartpole

Notebook ini adalah sebahagian daripada [Kurikulum AI untuk Pemula](http://aka.ms/ai-beginners). Ia diilhamkan oleh [tutorial rasmi PyTorch](https://pytorch.org/tutorials/intermediate/reinforcement_q_learning.html) dan [implementasi Cartpole PyTorch ini](https://github.com/yc930401/Actor-Critic-pytorch).

Dalam contoh ini, kita akan menggunakan RL untuk melatih model agar dapat menyeimbangkan tiang pada kereta yang boleh bergerak ke kiri dan kanan pada skala mendatar. Kita akan menggunakan persekitaran [OpenAI Gym](https://www.gymlibrary.ml/) untuk mensimulasikan tiang tersebut.

> **Nota**: Anda boleh menjalankan kod pelajaran ini secara tempatan (contohnya dari Visual Studio Code), di mana simulasi akan dibuka dalam tetingkap baru. Apabila menjalankan kod secara dalam talian, anda mungkin perlu membuat beberapa penyesuaian pada kod, seperti yang diterangkan [di sini](https://towardsdatascience.com/rendering-openai-gym-envs-on-binder-and-google-colab-536f99391cc7).

Kita akan bermula dengan memastikan Gym telah dipasang:


In [ ]:
import sys
!{sys.executable} -m pip install gym

Sekarang mari kita cipta persekitaran CartPole dan lihat bagaimana untuk mengendalikannya. Sesebuah persekitaran mempunyai ciri-ciri berikut:

* **Ruang tindakan** ialah set tindakan yang boleh kita lakukan pada setiap langkah simulasi
* **Ruang pemerhatian** ialah ruang pemerhatian yang boleh kita buat


In [ ]:
import gym

env = gym.make("CartPole-v1")

print(f"Action space: {env.action_space}")
print(f"Observation space: {env.observation_space}")

Mari kita lihat bagaimana simulasi ini berfungsi. Gelung berikut menjalankan simulasi sehingga `env.step` tidak lagi mengembalikan bendera penamatan `done`. Kita akan memilih tindakan secara rawak menggunakan `env.action_space.sample()`, yang bermaksud eksperimen ini mungkin akan gagal dengan sangat cepat (persekitaran CartPole akan tamat apabila kelajuan CartPole, posisinya, atau sudutnya berada di luar had tertentu).

> Simulasi akan dibuka dalam tetingkap baru. Anda boleh menjalankan kod ini beberapa kali dan melihat bagaimana ia berkelakuan.


In [ ]:
env.reset()

done = False
total_reward = 0
while not done:
   env.render()
   obs, rew, done, info = env.step(env.action_space.sample())
   total_reward += rew
   print(f"{obs} -> {rew}")
print(f"Total reward: {total_reward}")

Anda boleh perhatikan bahawa pemerhatian mengandungi 4 nombor. Nombor-nombor tersebut adalah:
- Kedudukan troli
- Kelajuan troli
- Sudut tiang
- Kadar putaran tiang

`rew` ialah ganjaran yang kita terima pada setiap langkah. Dalam persekitaran CartPole, anda akan menerima 1 mata untuk setiap langkah simulasi, dan matlamatnya adalah untuk memaksimumkan jumlah ganjaran, iaitu masa CartPole dapat menyeimbangkan tanpa jatuh.

Semasa pembelajaran pengukuhan, matlamat kita adalah untuk melatih **dasar** $\pi$, yang untuk setiap keadaan $s$ akan memberitahu kita tindakan $a$ yang perlu diambil, jadi pada asasnya $a = \pi(s)$.

Jika anda mahukan penyelesaian secara kebarangkalian, anda boleh menganggap dasar sebagai mengembalikan satu set kebarangkalian untuk setiap tindakan, iaitu $\pi(a|s)$ akan bermaksud kebarangkalian bahawa kita harus mengambil tindakan $a$ pada keadaan $s$.

## Kaedah Policy Gradient

Dalam algoritma RL yang paling mudah, dipanggil **Policy Gradient**, kita akan melatih rangkaian neural untuk meramalkan tindakan seterusnya.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import torch

num_inputs = 4
num_actions = 2

model = torch.nn.Sequential(
    torch.nn.Linear(num_inputs, 128, bias=False, dtype=torch.float32),
    torch.nn.ReLU(),
    torch.nn.Linear(128, num_actions, bias = False, dtype=torch.float32),
    torch.nn.Softmax(dim=1)
)

Kami akan melatih rangkaian dengan menjalankan banyak eksperimen, dan mengemas kini rangkaian kami selepas setiap percubaan. Mari kita definisikan satu fungsi yang akan menjalankan eksperimen dan mengembalikan hasilnya (dikenali sebagai **jejak**) - semua keadaan, tindakan (dan kebarangkalian yang disyorkan), dan ganjaran:


In [ ]:
def run_episode(max_steps_per_episode = 10000,render=False):    
    states, actions, probs, rewards = [],[],[],[]
    state = env.reset()
    for _ in range(max_steps_per_episode):
        if render:
            env.render()
        action_probs = model(torch.from_numpy(np.expand_dims(state,0)))[0]
        action = np.random.choice(num_actions, p=np.squeeze(action_probs.detach().numpy()))
        nstate, reward, done, info = env.step(action)
        if done:
            break
        states.append(state)
        actions.append(action)
        probs.append(action_probs.detach().numpy())
        rewards.append(reward)
        state = nstate
    return np.vstack(states), np.vstack(actions), np.vstack(probs), np.vstack(rewards)

Anda boleh menjalankan satu episod dengan rangkaian yang tidak terlatih dan perhatikan bahawa ganjaran keseluruhan (AKA panjang episod) adalah sangat rendah:


In [ ]:
s, a, p, r = run_episode()
print(f"Total reward: {np.sum(r)}")

Salah satu aspek sukar dalam algoritma polisi kecerunan adalah menggunakan **ganjaran terdiskaun**. Idea di sebaliknya adalah kita mengira vektor jumlah ganjaran pada setiap langkah permainan, dan semasa proses ini kita mendiskaunkan ganjaran awal menggunakan beberapa koefisien $gamma$. Kita juga menormalkan vektor yang terhasil, kerana kita akan menggunakannya sebagai berat untuk mempengaruhi latihan kita:


In [ ]:
eps = 0.0001

def discounted_rewards(rewards,gamma=0.99,normalize=True):
    ret = []
    s = 0
    for r in rewards[::-1]:
        s = r + gamma * s
        ret.insert(0, s)
    if normalize:
        ret = (ret-np.mean(ret))/(np.std(ret)+eps)
    return ret

Sekarang mari kita mulakan latihan sebenar! Kita akan menjalankan 300 episod, dan pada setiap episod kita akan melakukan perkara berikut:

1. Jalankan eksperimen dan kumpulkan jejak
1. Kira perbezaan (`gradients`) antara tindakan yang diambil dan kebarangkalian yang diramalkan. Semakin kecil perbezaannya, semakin yakin kita bahawa tindakan yang diambil adalah betul.
1. Kira ganjaran yang didiskaunkan dan darabkan `gradients` dengan ganjaran yang didiskaunkan - ini akan memastikan langkah dengan ganjaran yang lebih tinggi memberi kesan lebih besar pada hasil akhir berbanding langkah dengan ganjaran yang lebih rendah.
1. Tindakan sasaran yang dijangka untuk rangkaian neural kita sebahagiannya akan diambil daripada kebarangkalian yang diramalkan semasa eksperimen dijalankan, dan sebahagiannya daripada `gradients` yang dikira. Kita akan menggunakan parameter `alpha` untuk menentukan sejauh mana `gradients` dan ganjaran diambil kira - ini dipanggil *kadar pembelajaran* dalam algoritma pengukuhan.
1. Akhir sekali, kita melatih rangkaian kita berdasarkan keadaan dan tindakan yang dijangka, dan ulangi proses tersebut.


In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

def train_on_batch(x, y):
    x = torch.from_numpy(x)
    y = torch.from_numpy(y)
    optimizer.zero_grad()
    predictions = model(x)
    loss = -torch.mean(torch.log(predictions) * y)
    loss.backward()
    optimizer.step()
    return loss

In [ ]:
alpha = 1e-4

history = []
for epoch in range(300):
    states, actions, probs, rewards = run_episode()
    one_hot_actions = np.eye(2)[actions.T][0]
    gradients = one_hot_actions-probs
    dr = discounted_rewards(rewards)
    gradients *= dr
    target = alpha*np.vstack([gradients])+probs
    train_on_batch(states,target)
    history.append(np.sum(rewards))
    if epoch%100==0:
        print(f"{epoch} -> {np.sum(rewards)}")

plt.plot(history)

Sekarang mari jalankan episod dengan rendering untuk melihat hasilnya:


In [ ]:
_ = run_episode(render=True)

Semoga anda dapat melihat bahawa tiang kini boleh seimbang dengan agak baik!

## Model Actor-Critic

Model Actor-Critic adalah perkembangan lanjut daripada policy gradients, di mana kita membina rangkaian neural untuk mempelajari kedua-dua polisi dan ganjaran yang dianggarkan. Rangkaian ini akan mempunyai dua output (atau anda boleh melihatnya sebagai dua rangkaian berasingan):
* **Actor** akan mencadangkan tindakan yang perlu diambil dengan memberikan kita taburan kebarangkalian keadaan, seperti dalam model policy gradient.
* **Critic** akan menganggarkan apa ganjaran yang mungkin diperoleh daripada tindakan tersebut. Ia mengembalikan jumlah ganjaran yang dianggarkan pada masa hadapan dalam keadaan tertentu.

Mari kita definisikan model seperti ini:


In [ ]:
from itertools import count
import torch.nn.functional as F

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
env = gym.make("CartPole-v1")

state_size = env.observation_space.shape[0]
action_size = env.action_space.n
lr = 0.0001

class Actor(torch.nn.Module):
    def __init__(self, state_size, action_size):
        super(Actor, self).__init__()
        self.state_size = state_size
        self.action_size = action_size
        self.linear1 = torch.nn.Linear(self.state_size, 128)
        self.linear2 = torch.nn.Linear(128, 256)
        self.linear3 = torch.nn.Linear(256, self.action_size)

    def forward(self, state):
        output = F.relu(self.linear1(state))
        output = F.relu(self.linear2(output))
        output = self.linear3(output)
        distribution = torch.distributions.Categorical(F.softmax(output, dim=-1))
        return distribution


class Critic(torch.nn.Module):
    def __init__(self, state_size, action_size):
        super(Critic, self).__init__()
        self.state_size = state_size
        self.action_size = action_size
        self.linear1 = torch.nn.Linear(self.state_size, 128)
        self.linear2 = torch.nn.Linear(128, 256)
        self.linear3 = torch.nn.Linear(256, 1)

    def forward(self, state):
        output = F.relu(self.linear1(state))
        output = F.relu(self.linear2(output))
        value = self.linear3(output)
        return value

Kita perlu sedikit mengubah fungsi `discounted_rewards` dan `run_episode` kita:


In [ ]:
def discounted_rewards(next_value, rewards, masks, gamma=0.99):
    R = next_value
    returns = []
    for step in reversed(range(len(rewards))):
        R = rewards[step] + gamma * R * masks[step]
        returns.insert(0, R)
    return returns

def run_episode(actor, critic, n_iters):
    optimizerA = torch.optim.Adam(actor.parameters())
    optimizerC = torch.optim.Adam(critic.parameters())
    for iter in range(n_iters):
        state = env.reset()
        log_probs = []
        values = []
        rewards = []
        masks = []
        entropy = 0
        env.reset()

        for i in count():
            env.render()
            state = torch.FloatTensor(state).to(device)
            dist, value = actor(state), critic(state)

            action = dist.sample()
            next_state, reward, done, _ = env.step(action.cpu().numpy())

            log_prob = dist.log_prob(action).unsqueeze(0)
            entropy += dist.entropy().mean()

            log_probs.append(log_prob)
            values.append(value)
            rewards.append(torch.tensor([reward], dtype=torch.float, device=device))
            masks.append(torch.tensor([1-done], dtype=torch.float, device=device))

            state = next_state

            if done:
                print('Iteration: {}, Score: {}'.format(iter, i))
                break


        next_state = torch.FloatTensor(next_state).to(device)
        next_value = critic(next_state)
        returns = discounted_rewards(next_value, rewards, masks)

        log_probs = torch.cat(log_probs)
        returns = torch.cat(returns).detach()
        values = torch.cat(values)

        advantage = returns - values

        actor_loss = -(log_probs * advantage.detach()).mean()
        critic_loss = advantage.pow(2).mean()

        optimizerA.zero_grad()
        optimizerC.zero_grad()
        actor_loss.backward()
        critic_loss.backward()
        optimizerA.step()
        optimizerC.step()


Sekarang kita akan menjalankan gelung latihan utama. Kita akan menggunakan proses latihan rangkaian secara manual dengan mengira fungsi kehilangan yang sesuai dan mengemas kini parameter rangkaian:


In [ ]:

actor = Actor(state_size, action_size).to(device)
critic = Critic(state_size, action_size).to(device)
run_episode(actor, critic, n_iters=100)

In [ ]:
env.close()

## Intipati

Kita telah melihat dua algoritma RL dalam demo ini: simple policy gradient, dan actor-critic yang lebih canggih. Anda boleh lihat bahawa algoritma-algoritma tersebut beroperasi dengan konsep abstrak seperti keadaan, tindakan, dan ganjaran - oleh itu, ia boleh digunakan dalam persekitaran yang sangat berbeza.

Pembelajaran pengukuhan membolehkan kita mempelajari strategi terbaik untuk menyelesaikan masalah hanya dengan melihat ganjaran akhir. Hakikat bahawa kita tidak memerlukan dataset berlabel membolehkan kita mengulangi simulasi berkali-kali untuk mengoptimumkan model kita. Walau bagaimanapun, masih terdapat banyak cabaran dalam RL, yang mungkin anda pelajari jika anda memutuskan untuk memberi tumpuan lebih kepada bidang AI yang menarik ini.



---

**Penafian**:  
Dokumen ini telah diterjemahkan menggunakan perkhidmatan terjemahan AI [Co-op Translator](https://github.com/Azure/co-op-translator). Walaupun kami berusaha untuk memastikan ketepatan, sila ambil perhatian bahawa terjemahan automatik mungkin mengandungi kesilapan atau ketidaktepatan. Dokumen asal dalam bahasa asalnya harus dianggap sebagai sumber yang berwibawa. Untuk maklumat penting, terjemahan manusia profesional adalah disyorkan. Kami tidak bertanggungjawab atas sebarang salah faham atau salah tafsir yang timbul daripada penggunaan terjemahan ini.
